# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = [int(n) for n in lines[1].split() if n.isdigit()][0] #trova il primo numero
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[-1]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(" ")[-1].replace('\x80', ''))
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        return (ask_price * self.quantity)
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        global production_cost
        return (self.quantity * (ask_price - production_cost))
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
        
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    ######## SCRIVI CODICE ########
    file = open(f"{i+1}.txt", "r", encoding="ISO-8859-1") 
    #ho dovuto aggiungere encoding="ISO-8859-1" perchè mi dave errori nell'apertura dei files
    email = file.read()
    orders.append (Order(email))
    file.close()
    # Apri il file ... ordrs.append( ... Chiudi il file

In [4]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = [row.Profit(row.bid_price) for row in orders]
#[print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = orders [possible_profits.index(max(possible_profits))]

# Determina il prezzo migliore
best_price = max_profit.bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

#se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita,
#il cliente non acquisterà
my_orders_to_go = [ord1 for ord1 in orders if ord1.bid_price > best_price]
print (len(my_orders_to_go))

Il miglior prezzo è €19.85
Si eseguiranno 420 ordini
9


In [10]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
delivery_costs = [cost_less_100 * ord1.Distance() if ord1.quantity < 100 else cost_at_least_100 * ord1.Distance() for ord1 in orders]
for i in range(len(possible_profits)):
    possible_profits[i] += delivery_costs[i]
#[print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = orders [possible_profits.index(max(possible_profits))]


# Determina il prezzo migliore
best_price = max_profit.bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

######## SCRIVI CODICE ########
my_orders_to_go = [ord1 for ord1 in orders if ord1.bid_price > best_price]
print (len(my_orders_to_go))
######## SCRIVI CODICE ########
my_orders_to_go.sort(key =lambda otg: otg.Profit(best_price) )
[otg.Details() for otg in my_orders_to_go]

Il miglior prezzo è €19.85
Si eseguiranno 420 ordini
9
JP Iacovelli SPA ha ordinato 29 magliette taglia XXL a 19.96€ all'indirizzo [46.035413, 6.990662]
JP Moneta e Sorelle ha ordinato 55 magliette taglia L a 19.93€ all'indirizzo [45.677897, 7.120169]
Francesco Iacovelli SPA ha ordinato 245 magliette taglia L a 19.87€ all'indirizzo [46.633524, 12.232977]
JP Cerisola Giochi d'Azzardo ha ordinato 156 magliette taglia L a 19.86€ all'indirizzo [45.337628, 9.235005]
MC Zacchei Latte ha ordinato 351 magliette taglia XS a 19.88€ all'indirizzo [45.498935, 8.319235]
Alessandro Bressan IB ha ordinato 242 magliette taglia XS a 19.99€ all'indirizzo [45.152525, 8.134165]
Filippo Zacchei SRL ha ordinato 353 magliette taglia S a 19.91€ all'indirizzo [45.442417, 11.506261]
Carlo Augusto Cerisola e Fratelli ha ordinato 135 magliette taglia XS a 19.88€ all'indirizzo [46.429483, 11.714962]
JP Carrara Servizi Finanziari ha ordinato 486 magliette taglia XS a 19.87€ all'indirizzo [46.177794, 8.410633]


[None, None, None, None, None, None, None, None, None]